In [1]:
import relbench
from relbench.base import Table, Database, Dataset, EntityTask
from relbench.datasets import get_dataset
from relbench.tasks import get_task
from relbench.base import BaseTask
from torch_geometric.seed import seed_everything
from relbench.modeling.utils import get_stype_proposal
from relbench.modeling.graph import make_pkey_fkey_graph
from relbench.modeling.graph import get_node_train_table_input
from torch_geometric.loader import NeighborLoader

import os
import math
import numpy as np
from tqdm import tqdm

import torch
from torch import Tensor
import torch_geometric
import torch_frame

from torch_frame.config.text_embedder import TextEmbedderConfig
from typing import List, Optional
from sentence_transformers import SentenceTransformer

from torch.nn import L1Loss
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

seed_everything(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [2]:
dataset = get_dataset(name="rel-trial", download=True)
db = dataset.get_db()
task_a = get_task("rel-trial", "study-outcome", download = True)
task_b = get_task("rel-trial", "site-success", download = True)

Loading Database object from /home/lingze/.cache/relbench/rel-trial/db...
Done in 8.15 seconds.


In [3]:
col_to_stype_dict = get_stype_proposal(db)

class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device
                                       ] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return torch.from_numpy(self.model.encode(sentences))

text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device), batch_size=256
)


root_dir = "/home/lingze/embedding_fusion/data"
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,  # speficied column types
    text_embedder_cfg=text_embedder_cfg,  # our chosen text encoder
    cache_dir=os.path.join(
        root_dir, f"rel-trial_materialized_cache"
    ),  # store materialized graph for convenience
)

/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
def generate_loader_dict(task: BaseTask) -> dict:
    loader_dict = {}
    for split, table in [
        ("train", task.get_table("train")),
        ("val",task.get_table("val")),
        ("test", task.get_table("test")),
    ]:
        table_input = get_node_train_table_input(
            table=table,
            task=task,
        )
        loader_dict[split] = NeighborLoader(
            data,
            num_neighbors=[
                128 for i in range(2)
            ],  # we sample subgraphs of depth 2, 128 neighbors per node.
            time_attr="time",
            input_nodes=table_input.nodes,
            input_time=table_input.time,
            transform=table_input.transform,
            batch_size=512,
            temporal_strategy="uniform",
            shuffle=split == "train",
            num_workers=0,
            persistent_workers=False,
        )
    return loader_dict

In [5]:
taska_loader_dict = generate_loader_dict(task_a)
taskb_loader_dict = generate_loader_dict(task_b)

In [6]:
# declare the model
from torch.nn import BCEWithLogitsLoss
import copy
from typing import Any, Dict, List, Optional

import torch
import torch_frame
from torch import Tensor
from torch.nn import Embedding, ModuleDict
from torch_frame.data.stats import StatType
from torch_geometric.data import HeteroData
from torch_geometric.nn import MLP
from torch_frame.nn.models.resnet import FCResidualBlock
from torch_geometric.typing import NodeType
from torch_frame.nn.encoder import StypeWiseFeatureEncoder
from relbench.modeling.nn import HeteroEncoder, HeteroGraphSAGE, HeteroTemporalEncoder

In [7]:
class FeatureEncodingPart(torch.nn.Module):
    def __init__(
        self,
        data:HeteroData,
        node_to_col_stats: Dict[str, Dict[str, Dict[StatType, Tensor]]],
        channels: int
    ):
        super().__init__()
        self.encoders = torch.nn.ModuleDict()
        # node_type : StypeWiseFeatureEncoder
        
        node_to_col_names_dict = {
            node_type: data[node_type].tf.col_names_dict
            for node_type in data.node_types
        }
        # node_type:  {stype: [col_name]}
        
        default_stype_encoder_cls_kwargs: Dict[torch_frame.stype, Any] = {
            torch_frame.categorical: (torch_frame.nn.EmbeddingEncoder, {}),
            torch_frame.numerical: (torch_frame.nn.LinearEncoder, {}),
            torch_frame.multicategorical: (
                torch_frame.nn.MultiCategoricalEmbeddingEncoder,
                {},
            ),
            torch_frame.embedding: (torch_frame.nn.LinearEmbeddingEncoder, {}),
            torch_frame.timestamp: (torch_frame.nn.TimestampEncoder, {}),
        }
                
        for node_type in node_to_col_names_dict.keys():
            stype_encoder_dict = {
                stype: default_stype_encoder_cls_kwargs[stype][0](
                    **default_stype_encoder_cls_kwargs[stype][1]
                )
                for stype in node_to_col_names_dict[node_type].keys()
            }
            self.encoders.update({node_type: StypeWiseFeatureEncoder(
                out_channels=channels,
                col_stats = node_to_col_stats[node_type],
                col_names_dict=node_to_col_names_dict[node_type],
                stype_encoder_dict=stype_encoder_dict
            )})
    
    def reset_parameters(self):
        for encoder in self.encoders.values():
            encoder.reset_parameters()
    
    def forward(
        self,
        tf_dict: Dict[NodeType, torch_frame.TensorFrame],
    )-> Dict[NodeType, Tensor]:
        x_dict = {}
        for node_type, tf in tf_dict.items():
            x, _ = self.encoders[node_type](tf)
            x_dict[node_type] = x
        return x_dict

In [8]:
class NodeRepresentationPart(torch.nn.Module):
    def __init__(
        self,
        data: HeteroData,
        channels:int,
        num_layers: int,
        normalization: Optional[str] = "layer_norm",
        dropout_prob: float = 0.0
    ):
        super().__init__()
        
        self.mappers = torch.nn.ModuleDict()
        
        node_to_col_names_dict = {
            node_type: data[node_type].tf.col_names_dict
            for node_type in data.node_types
        } 
        # node_type:  {stype: [col_name]}
        
        for node_type, type_to_col_names in node_to_col_names_dict.items():
            col_cnt = 0
            for cols in type_to_col_names.values():
                col_cnt += len(cols)
            in_channels = col_cnt * channels
            backbone = torch.nn.Sequential(*[
                FCResidualBlock(
                    in_channels if i == 0 else channels,
                    channels,
                    normalization=normalization,
                    dropout_prob=dropout_prob
                )
            for i in range(num_layers)], 
            torch.nn.LayerNorm(channels),
            torch.nn.ReLU(),
            torch.nn.Linear(channels, channels)
            )
            self.mappers.update(
                {
                    node_type: backbone
                }
            )
    def reset_parameters(self):
        for mapper in self.mappers.values():
            for layer in mapper:
                if hasattr(layer, "reset_parameters"):
                    layer.reset_parameters()
    
    def forward(self, 
                x_dict: Dict[NodeType, Tensor]
        ) -> Dict[NodeType, Tensor]:
        out_dict = {}
        for node_type, x in x_dict.items():
            # Flattening the encoder output
            x = x.view(x.size(0), math.prod(x.shape[1:]))
            out_dict[node_type] = self.mappers[node_type](x)
        return out_dict

In [9]:
class CompositeModel(torch.nn.Module):
    
    def __init__(
        self,
        data: HeteroData,
        channels: int,
        out_channels:int,
        aggr:str,
        norm:str,
        num_layer:int,
        feature_encoder: torch.nn.Module,
        node_encoder: torch.nn.Module,
        temporal_encoder: torch.nn.Module,
    ):
        super().__init__()
        self.gnn = HeteroGraphSAGE(
            node_types = data.node_types,
            edge_types= data.edge_types,
            channels=channels,
            aggr = aggr,
            num_layers=num_layer
        )
        
        self.head = MLP(
            channels,
            out_channels=out_channels,
            norm=norm,
            num_layers=1
        )
        
        self.feature_encoder = feature_encoder
        self.node_encoder = node_encoder
        self.temporal_encoder = temporal_encoder
    
    
    def reset_parameters(self):
        self.gnn.reset_parameters()
        self.head.reset_parameters()
    
    def forward(
        self,
        batch: HeteroData,
        entity_table: NodeType
    ) -> Tensor:
        seed_time = batch[entity_table].seed_time
        x_dict = self.feature_encoder(batch.tf_dict)
        x_dict = self.node_encoder(x_dict)
        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )
        
        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time
        
        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
            batch.num_sampled_nodes_dict,
            batch.num_sampled_edges_dict,
        )
        return self.head(x_dict[entity_table][: seed_time.size(0)]) 

In [27]:
@torch.no_grad()
def valid(loader: NeighborLoader, model: torch.nn.Module, task: BaseTask)-> np.ndarray:
    model.eval()
    pred_list = []
    pred_hat_list = []
    for batch in loader:
        batch = batch.to(device)
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
        pred_hat_list.append(batch[task.entity_table].y.detach().cpu())
    return torch.cat(pred_list, dim=0), torch.cat(pred_hat_list, dim=0)

@torch.no_grad()
def test(loader: NeighborLoader, model: torch.nn.Module, task: BaseTask)-> np.ndarray:
    model.eval()
    pred_list = []
    for batch in loader:
        batch = batch.to(device)
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
    return torch.cat(pred_list, dim=0)

In [26]:
channels = 128

task_a_temporal_encoder = HeteroTemporalEncoder(
    node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
            channels=channels,
)

task_a_feat_encoder = FeatureEncodingPart(
    data=data,
    node_to_col_stats=col_stats_dict,
    channels=channels
)


task_a_node_encoder = NodeRepresentationPart(
    data=data,
    channels=channels,
    num_layers=1,
    normalization="layer_norm",
    dropout_prob=0.2
)



task_a_model =  CompositeModel(
    data=data,
    channels=channels,
    out_channels=1,
    aggr="mean",
    norm="layer_norm",
    num_layer=2,
    feature_encoder=task_a_feat_encoder,
    node_encoder=task_a_node_encoder,
    temporal_encoder=task_a_temporal_encoder
)


In [28]:
# Train Task A
optimizer = torch.optim.Adam(task_a_model.parameters(), lr=0.005)
epochs = 15
loss_fn = BCEWithLogitsLoss()
tune_metric = "auroc"
higher_is_better = True


In [29]:
state_dict = None
best_val_metric = -math.inf if higher_is_better else math.inf
task_a_model.to(device)
best_epoch = 0
task_a_model.reset_parameters()
for epoch in range(1, epochs + 1):
    task_a_model.train()
    loss_accum = count_accum = 0
    for batch in tqdm(taska_loader_dict["train"]):
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = task_a_model(
            batch,
            task_a.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        loss = loss_fn(pred, batch[task_a.entity_table].y.float())
        
        loss.backward()
        optimizer.step()
        
        loss_accum += loss.detach().item() * pred.size(0)
        count_accum += pred.size(0)
    
    train_loss = loss_accum / count_accum
    val_logits = test(taska_loader_dict["val"], task_a_model, task_a)
    val_logits = torch.sigmoid(val_logits).numpy()
    
    val_pred = (val_logits > 0.5).astype(int)
    val_pred_hat = task_a.get_table("val").df[task_a.target_col].to_numpy()
    val_metrics = {
            "auroc": roc_auc_score(val_pred_hat, val_logits),
        "accuracy": accuracy_score(val_pred_hat, val_pred),
        "precision": precision_score(val_pred_hat, val_pred),
        "recall": recall_score(val_pred_hat, val_pred),
        "f1": f1_score(val_pred_hat, val_pred),
    }
    
    print(f"Epoch: {epoch:02d}, Train loss: {train_loss}, Val metrics: {val_metrics}")

    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
        not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        best_epoch = epoch
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(task_a_model.state_dict())

best_epoch

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:03<00:00,  6.05it/s]


Epoch: 01, Train loss: 0.6837852612244639, Val metrics: {'auroc': 0.5895911793744611, 'accuracy': 0.584375, 'precision': 0.584375, 'recall': 1.0, 'f1': 0.73767258382643}


100%|██████████| 24/24 [00:03<00:00,  6.19it/s]


Epoch: 02, Train loss: 0.6419359451336564, Val metrics: {'auroc': 0.6175353714053404, 'accuracy': 0.584375, 'precision': 0.584375, 'recall': 1.0, 'f1': 0.73767258382643}


...

100%|██████████| 24/24 [00:03<00:00,  6.28it/s]


Epoch: 14, Train loss: 0.43261125400023837, Val metrics: {'auroc': 0.6264457936284562, 'accuracy': 0.60625, 'precision': 0.6414219474497682, 'recall': 0.7397504456327986, 'f1': 0.6870860927152318}


100%|██████████| 24/24 [00:03<00:00,  6.37it/s]


Epoch: 15, Train loss: 0.42252855523439253, Val metrics: {'auroc': 0.6376458079244457, 'accuracy': 0.6145833333333334, 'precision': 0.6547811993517018, 'recall': 0.7201426024955436, 'f1': 0.6859083191850595}


4

In [31]:
# test task A
task_a_model.load_state_dict(state_dict)
test_logits = test(taska_loader_dict["test"], task_a_model, task_a)
test_logits =  torch.sigmoid(test_logits).numpy()

test_pred = (test_logits > 0.5).astype(int)
test_pred_hat = task_a.get_table("test", mask_input_cols = False).df[task_a.target_col].to_numpy()
test_metrics = {
    "auroc": roc_auc_score(test_pred_hat, test_logits),
    "accuracy": accuracy_score(test_pred_hat, test_pred),
    "precision": precision_score(test_pred_hat, test_pred),
    "recall": recall_score(test_pred_hat, test_pred),
    "f1score": f1_score(test_pred_hat, test_pred),
}
test_metrics

{'auroc': 0.7006465438959718,
 'accuracy': 0.6472727272727272,
 'precision': 0.6486068111455109,
 'recall': 0.8674948240165632,
 'f1score': 0.7422497785651019}

In [ ]:
# train Task B

In [50]:
task_b_temporal_encoder = HeteroTemporalEncoder(
    node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
            channels=channels,
)

task_b_feat_encoder = FeatureEncodingPart(
    data=data,
    node_to_col_stats=col_stats_dict,
    channels=channels
)


task_b_node_encoder = NodeRepresentationPart(
    data=data,
    channels=channels,
    num_layers=1,
    normalization="layer_norm",
    dropout_prob=0.3
)

task_b_model =  CompositeModel(
    data=data,
    channels=channels,
    out_channels=1,
    aggr="mean",
    norm="layer_norm",
    num_layer=2,
    feature_encoder=task_b_feat_encoder,
    node_encoder=task_b_node_encoder,
    temporal_encoder=task_b_temporal_encoder
)

In [53]:
optimizer = torch.optim.Adam(task_b_model.parameters(), lr=0.002)
epochs = 20
loss_fn = L1Loss()
tune_metric = "mae"
higher_is_better = False

In [54]:
state_dict = None
best_val_metric = -math.inf if higher_is_better else math.inf
task_b_model.to(device)
best_epoch = 0
early_stop = 20
# train
for epoch in range(1, epochs + 1):
    task_b_model.train()
    
    cnt = 0
    loss_accum = count_accum = 0
    for batch in tqdm(taskb_loader_dict["train"]):
        cnt += 1
        if cnt > early_stop:
            break
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = task_b_model(
            batch,
            task_b.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        loss = loss_fn(pred, batch[task_b.entity_table].y.float())
        
        loss.backward()
        optimizer.step()
        
        loss_accum += loss.detach().item() * pred.size(0)
        count_accum += pred.size(0)

    train_loss = loss_accum / count_accum
    val_pred_hat = task_b.get_table("val").df[task_b.target_col].to_numpy()
    val_logits = test(taskb_loader_dict["val"], task_b_model, task_b)
    val_logits = val_logits.numpy()
    
    val_metrics = {
        "mae": mean_absolute_error(val_pred_hat, val_logits),
        "r2": r2_score(val_pred_hat, val_logits),
        "rmse": root_mean_squared_error(val_pred_hat, val_logits),
    }
    
    print(f"Epoch: {epoch:02d}, Train loss: {train_loss}, Val metrics: {val_metrics}")

    
    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
        not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        best_epoch = epoch
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(task_b_model.state_dict())

best_epoch

  7%|▋         | 20/296 [00:02<00:32,  8.39it/s]


Epoch: 01, Train loss: 0.41037669628858564, Val metrics: {'mae': 0.4475658469513964, 'r2': -0.4558490914968616, 'rmse': 0.5763787357807583}


  7%|▋         | 20/296 [00:02<00:29,  9.35it/s]


Epoch: 02, Train loss: 0.3218929409980774, Val metrics: {'mae': 0.43447137506095423, 'r2': -0.4276799351072853, 'rmse': 0.5707753357201678}


  7%|▋         | 20/296 [00:02<00:31,  8.69it/s]


...

  7%|▋         | 20/296 [00:02<00:32,  8.41it/s]


Epoch: 19, Train loss: 0.2103615455329418, Val metrics: {'mae': 0.40700702387336735, 'r2': -0.4448564994177324, 'rmse': 0.574198598516727}


  7%|▋         | 20/296 [00:02<00:32,  8.39it/s]


Epoch: 20, Train loss: 0.20522889345884324, Val metrics: {'mae': 0.4036564826480426, 'r2': -0.3692540406407825, 'rmse': 0.558974229278604}


15

In [55]:
# test
task_b_model.load_state_dict(state_dict)
logits = test(taskb_loader_dict["test"], task_b_model, task_b)
logits = logits.numpy()
pred_hat = task_b.get_table("test", mask_input_cols=False).df[task_b.target_col].to_numpy()
test_metrics = {
        "mae": mean_absolute_error(pred_hat, logits),
        "r2": r2_score(pred_hat, logits),
        "rmse": root_mean_squared_error(pred_hat, logits),
}
test_metrics

{'mae': 0.4027192653319429,
 'r2': -0.43432623996859543,
 'rmse': 0.5763256675733064}

: 